In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import urllib
import os
import json

%matplotlib inline

In [2]:
## A dict with sources:
with open('sources.json') as data_file:    
    sources = json.load(data_file)

In [3]:
def clean_index_item(item):
    # Some empty index columns:
    if type(item) == float:
        return item
    
    # A lot of random extra spaces in index columns
    item = item.strip()
    
    # Asterisks too
    item = filter(lambda l: l!="*", item)
    
    # Inconsistent naming
    item = item.replace("Rep.", "Republic")
    item = item.replace("Balears", "Baleares")
    
    return item

def get_series_from_mecd(url, args):

    # Here I'm truncating the documentID in mecd.gob.es; hope
    # it doesn't bite me.
    temp_file = "data/%s.xlsx" % url[-12:]
  
    # Don't download if we already have the file
    # TODO: create folder if it doesn't exist
    if not os.path.isfile(temp_file):
        urllib.urlretrieve(url, temp_file)
        
    # Squeeze to get a Series if only one column
    series = pd.read_excel(temp_file, squeeze=True, **args)    

    # Do some cleaning on the index
    series.index = [clean_index_item(item) for item in series.index]
     
    # Discard non-numeric and NaN values
    series = pd.to_numeric(series, 'coerce')
    series = series[pd.notnull(series)]
    
    
    return series

In [4]:
series = {name: get_series_from_mecd(*values) for name, values in sources.iteritems()}
    


In [5]:
regions = {k: v for k, v in series.iteritems() if "regions" in k}
pd.DataFrame(regions).head()

,escs_regions,math_bottom_escs_regions,math_top_escs_regions,repeaters_regions,resilient_regions
Andalusia,15.968206,434.974162,521.416508,36.738454,4.997563
Aragon,15.656241,441.751113,538.642403,36.098311,4.175541
Asturias,15.601660,455.417450,550.482590,27.438242,3.660023
Balearic Islands,12.456936,432.179918,512.581851,39.145251,5.257534
Basque Country,10.478751,472.248973,541.621544,20.761495,2.025511


In [6]:
countries = {k: v for k, v in series.iteritems() if "countries" in k}
pd.DataFrame(countries).head()

,escs_countries,math_bottom_escs_countries,math_top_escs_countries,repeaters_countries,resilient_countries
Argentina,15.081970,NaN,NaN,NaN,NaN
Australia,12.348634,462.543084,549.585074,0.074888,1.174820
Austria,15.820689,458.067818,551.907495,0.119181,0.889132
Belgium,19.626139,460.120954,574.638110,0.361050,1.260430
Brazil,15.715390,359.574956,436.873628,NaN,0.071114


In [7]:
other = {k: v for k, v in series.iteritems() if "countries" not in k and "regions" not in k }
pd.DataFrame(other).head()

,math_level_1,math_level_5,math_level_6,math_migrant_grade,math_native_grade,math_score,math_under_1,reading_level_1,reading_level_5,reading_level_6,reading_score,reading_under_1,science_level_1,science_level_5,science_level_6,science_score,science_under_1
Alemania,12.193542,12.773757,4.681767,472.225526,527.969956,513.525056,5.545130,3.266599,24.577259,8.260698,507.676530,0.521725,9.312185,10.574336,1.590635,524.120799,2.902123
Andalucía,18.915585,4.732912,1.013611,435.112072,474.977152,472.019591,8.478546,5.178162,16.154161,3.984725,476.812111,1.643554,14.102856,4.116931,0.300497,486.034650,4.516763
Aragón,13.386463,9.498153,1.957469,426.412815,512.153297,496.266194,7.881926,4.429627,19.945232,5.932133,493.141800,1.096787,11.864777,6.762451,0.842837,504.111080,3.938644
Asturias,11.890078,10.387162,2.444741,431.404705,507.384719,499.601625,6.777473,3.674848,23.231226,7.373033,504.016976,1.305270,8.927664,8.123267,1.236010,516.861585,2.731494
Australia,13.534663,10.515042,4.294680,529.206051,501.606806,504.150766,6.132482,3.106353,23.336504,9.798264,511.803998,0.900302,10.214973,10.912487,2.644107,521.494746,3.430821
